In [25]:
import pandas as pd
import json
import numpy as np

In [16]:
raw_zip = json.load(open('data/zip_code_list.json'))

In [41]:
zip_df = pd.DataFrame(raw_zip, columns =['raw_zip'])
zip_df['adj_zip'] = 0
zip_df['country'] = 0
zip_df['timezone'] = 0
zip_df['longitude'] = 0
zip_df['latitude'] = 0
zip_df

,raw_zip,adj_zip,country,timezone,longitude,latitude
0,NaN,0,0,0,0,0
1,32928,0,0,0,0,0
2,07010-3107,0,0,0,0,0
3,91730-5743,0,0,0,0,0
4,65085,0,0,0,0,0
...,...,...,...,...,...,...
78530,497791331,0,0,0,0,0
78531,95666-9698,0,0,0,0,0
78532,40216-2302,0,0,0,0,0
78533,945016365,0,0,0,0,0


In [42]:
def set_country(row):
    # null cases
    if pd.isna(row["raw_zip"]) == True:
        return np.nan
    
    # when zip contains only numbers
    elif row["raw_zip"].isnumeric() == True:
        return len(row["raw_zip"])
    
    # when zip contains '-'
    elif ("-" in row["raw_zip"]) == True:
        return len(row["raw_zip"].split("-")[0])
    
    # other cases: when zip contains characters and special symbols
    else:
        return 0

zip_df = zip_df.assign(country=zip_df.apply(set_country, axis=1))

In [57]:
def set_zip(row):
    if (row["country"] == 5.0) == True:
        if ("-" in row["raw_zip"]) == True:
            return row["raw_zip"].split("-")[0]
        else:
            return row["raw_zip"]

zip_df = zip_df.assign(adj_zip=zip_df.apply(set_zip, axis=1))

In [58]:
zip_df

,raw_zip,adj_zip,country,timezone,longitude,latitude
0,NaN,None,NaN,0,0,0
1,32928,32928,5.0,0,0,0
2,07010-3107,07010,5.0,0,0,0
3,91730-5743,91730,5.0,0,0,0
4,65085,65085,5.0,0,0,0
...,...,...,...,...,...,...
78530,497791331,None,9.0,0,0,0
78531,95666-9698,95666,5.0,0,0,0
78532,40216-2302,40216,5.0,0,0,0
78533,945016365,None,9.0,0,0,0


In [81]:
# read external data file
ref_zip = pd.read_csv('data/uszips.csv')
#  补齐0，变成5位数
ref_zip['zip'] = ref_zip['zip'].astype(str).str.zfill(5)
ref_zip[:5]

,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone
0,00601,18.18005,-66.75218,Adjuntas,PR,Puerto Rico,True,NaN,17113.0,102.7,72001,Adjuntas,"{""72001"": ""99.43"", ""72141"": ""0.57""}",Adjuntas|Utuado,72001|72141,False,False,America/Puerto_Rico
1,00602,18.36074,-67.17519,Aguada,PR,Puerto Rico,True,NaN,37751.0,476.0,72003,Aguada,"{""72003"": ""100""}",Aguada,72003,False,False,America/Puerto_Rico
2,00603,18.45440,-67.12201,Aguadilla,PR,Puerto Rico,True,NaN,47081.0,574.9,72005,Aguadilla,"{""72005"": ""100""}",Aguadilla,72005,False,False,America/Puerto_Rico
3,00606,18.16721,-66.93828,Maricao,PR,Puerto Rico,True,NaN,6392.0,58.3,72093,Maricao,"{""72093"": ""94.88"", ""72153"": ""3.78"", ""72121"": ""...",Maricao|Yauco|Sabana Grande,72093|72153|72121,False,False,America/Puerto_Rico
4,00610,18.29032,-67.12244,Anasco,PR,Puerto Rico,True,NaN,26686.0,286.9,72011,Añasco,"{""72011"": ""99.45"", ""72003"": ""0.55""}",Añasco|Aguada,72011|72003,False,False,America/Puerto_Rico


In [85]:
merge_zip = zip_df.merge(ref_zip, how='left',left_on='adj_zip',right_on='zip',suffixes=('_left', '_right'))
merge_zip.drop('timezone_left', axis=1, inplace=True)
merge_zip.drop('longitude',axis=1, inplace=True)
merge_zip.drop('latitude', axis=1, inplace = True)
merge_zip

,raw_zip,adj_zip,country,zip,lat,lng,city,state_id,state_name,zcta,...,population,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone_right
0,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,32928,32928,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,07010-3107,07010,5.0,07010,40.82211,-73.98794,Cliffside Park,NJ,New Jersey,True,...,25126.0,10162.9,34003.0,Bergen,"{""34003"": ""100""}",Bergen,34003,False,False,America/New_York
3,91730-5743,91730,5.0,91730,34.09955,-117.57847,Rancho Cucamonga,CA,California,True,...,72407.0,1930.1,6071.0,San Bernardino,"{""06071"": ""100""}",San Bernardino,06071,False,False,America/Los_Angeles
4,65085,65085,5.0,65085,38.40179,-92.07865,Westphalia,MO,Missouri,True,...,1252.0,8.6,29151.0,Osage,"{""29151"": ""100""}",Osage,29151,False,False,America/Chicago
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78530,497791331,None,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78531,95666-9698,95666,5.0,95666,38.53205,-120.36845,Pioneer,CA,California,True,...,5164.0,35.8,6005.0,Amador,"{""06005"": ""100""}",Amador,06005,False,False,America/Los_Angeles
78532,40216-2302,40216,5.0,40216,38.18765,-85.83979,Louisville,KY,Kentucky,True,...,42029.0,1122.5,21111.0,Jefferson,"{""21111"": ""100""}",Jefferson,21111,False,False,America/Kentucky/Louisville
78533,945016365,None,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
merge_zip.count()

raw_zip             78534
adj_zip             72047
country             78534
zip                 62747
lat                 62747
lng                 62747
city                62747
state_id            62747
state_name          62747
zcta                62747
parent_zcta             0
population          62707
density             62707
county_fips         62747
county_name         62747
county_weights      62747
county_names_all    62747
county_fips_all     62747
imprecise           62747
military            62747
timezone_right      62747
dtype: int64